![ssss](http://cache.boston.com/universal/site_graphics/blogs/bigpicture/lhc_08_01/lhc1.jpg)

# Please wait few seconds to display graphs/images because plotly is little slow.
## More To Come !! Stayed Tuned.

- <a href='#1'>1. Introduction</a>
- <a href='#2'>2. Importing required libraries and data</a>
- <a href='#3'>3. Event hits Data</a>
    - <a href='#3-1'>3.1 Glimpse of Event hit data</a>
    - <a href='#3-2'>3.2 Vertical Intersection ( x,y ) in Detection Layers</a>
    - <a href='#3-3'>3.3 2D scatter plot </a>
    - <a href='#3-4'>3.4 2D Histogram</a>
    - <a href='#3-5'>3.5 3D Surface plot</a>
    - <a href='#3-6'>3.6 3D Sactter plot</a>
    - <a href='#3-7'>3.7 Horizontal Intersection ( z ,y ) in Detection Layers</a>
    - <a href='#3-8'>3.8 Distribution of Volume id, Layer id and Module id for single event<</a>
    - <a href='#3-9'>3.9 ScatterPlotMatrix of full hits table</a>
- <a href='#4'>4. Event hit cells data</a>
    - <a href='#4-1'>4.1 Glimpse of Event hit cells data</a>
    - <a href='#4-2'>4.2 Visualization of channel identifier/coordinates unique within one module.</a>
    - <a href='#4-3'>4.3 Channel identifiers (ch0, ch1) and signal value information for a single event</a>
    - <a href='#4-4'>4.4 ScatterPlotMatrix of Event hits cells</a> 
- <a href='#5'>5. Event particles Data </a>
    - <a href='#5-1'>5.1 Glimpse of Event particles data</a>
    - <a href='#5-2'>5.2 Initial position or vertex (in millimeters) in global coordinates. 'X' and 'Y' axis with number of hits generated by particular particle</a>
    - <a href='#5-3'>5.3 Distribution of particle charges in event000001005</a>
    - <a href='#5-4'>5.4 initial position or vertex (in millimeters) in global coordinates. 'X' and 'Y' axis with particle charge(positive or negative)</a>
    - <a href='#5-5'>5.5 Initial momentum (in GeV/c) along 'X' and 'Y' axis with particle charges(Positive or Negative).</a>
    - <a href='#5-6'>5.6 3D scatter plot of inintial momentum and initial vertex</a>
    - <a href='#5-7'>5.7 Initial position or vertex (in millimeters) in global coordinates. 'Z' and 'Y' axis with number of hits generated by particular particle</a>
    - <a href='#5-8'>5.8 initial momentum (in GeV/c) along 'X' and 'Y' global axis with number of hits and physics particles</a>
    - <a href='#5-9'>5.9 Distribution of number of hits generated by particular particle</a>
- <a href='#6'>6. Event truth data</a>
    - <a href='#6-1'>6.1 Glimpse of Event truth data</a>
    - <a href='#6-2'>6.2 Vertical Intersection ( x,y ) between the particle trajectory and the sensitive surface</a>
    - <a href='#6-3'>6.3 Horizontal Intersection ( z,y ) between the particle trajectory and the sensitive surface</a>
    - <a href='#6-4'>6.4 3D scatter plot of true particle momentum (in GeV/c) in the global coordinate system at the intersection point</a>
    - <a href='#6-5'>6.5 Distribution of Hit ID, Particle ID and weight for each measurement hit</a>
- <a href='#7'>7. Detector Geometry Information data</a>
    - <a href='#7-1'>7.1 Glimpse of  Detector Geometry Information data</a>
    - <a href='#7-2'>7.2 Distribution of module_t (thickness of the detector module (in millimeter)).</a>
    - <a href='#7-3'>7.3 Distribution of module_hv ( the half-length of the module boundary along the local v direction (in millimeter)).</a>
    - <a href='#7-4'>7.4 Correlation of features</a>
    - <a href='#7-5'>7.5 Distribution of Layer ID, Volume ID and Module ID of detector Geometry information data</a>
    - <a href='#7-6'>7.6  Distribution of the size of detector cells along the local u and v direction (in millimeter)</a>

# <a id='1'>1. Introduction</a>

To explore what our universe is made of, scientists at CERN are colliding protons, essentially recreating mini big bangs, and meticulously observing these collisions with intricate silicon detectors.

While orchestrating the collisions and observations is already a massive scientific accomplishment, analyzing the enormous amounts of data produced from the experiments is becoming an overwhelming challenge.

Event rates have already reached hundreds of millions of collisions per second, meaning physicists must sift through tens of petabytes of data per year. And, as the resolution of detectors improve, ever better software is needed for real-time pre-processing and filtering of the most promising events, producing even more data.

To help address this problem, a team of Machine Learning experts and physics scientists working at CERN (the world largest high energy physics laboratory), has partnered with Kaggle and prestigious sponsors to answer the question: can machine learning assist high energy physics in discovering and characterizing new particles?

Specifically, in this competition, you’re challenged to build an algorithm that quickly reconstructs particle tracks from 3D points left in the silicon detectors. 

# <a id='2'>2. Importing required libraries and data</a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import seaborn
import seaborn as sns

# import plotly and offline mode
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.figure_factory as ff

# import cufflinks and offline mode
import cufflinks as cf
cf.go_offline()

% matplotlib inline

# # for 3D plotting
# import ipyvolume as ipv
# import ipywidgets as widgets

from trackml.dataset import load_event, load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event

In [ ]:
hits, cells, particles, truth = load_event('../input/train_1/event000001005')
detectors = pd.read_csv("../input/detectors.csv")

# <a id='3'>3. Event hits Data</a>

## <a id='3-1'>3.1 Glimpse of Event hit data</a>

In [ ]:
hits.head()

The hits file contains the following values for each hit/entry:

- **hit_id:** numerical identifier of the hit inside the event.
- **x, y, z:** measured x, y, z position (in millimeter) of the hit in global coordinates.
- **volume_id:** numerical identifier of the detector group.
- **layer_id:** numerical identifier of the detector layer inside the group.
- **module_id:** numerical identifier of the detector module inside the layer.

The volume/layer/module id could in principle be deduced from x, y, z. They are given here to simplify detector-specific data handling.

## <a id='3-2'>3.2 Vertical Intersection ( x,y ) in Detection Layers</a>

In [ ]:
hits_samples = hits.sample(8000)

fig = ff.create_facet_grid(
    hits_samples,
    x='x',
    y='y',
    color_name='volume_id',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

In [ ]:
hits_samples = hits.sample(8000)

fig = ff.create_facet_grid(
    hits_samples,
    x='x',
    y='y',
    facet_col='volume_id',
    color_name='volume_id',
    color_is_cat=True,
#     color_name='volume_id',
#     show_boxes=False,
#     marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='3-3'>3.3 2D scatter plot </a>

In [ ]:
hits_samples = hits.sample(8000)
hits_samples.iplot(kind='scatter',x='x',y='y',mode='markers',size=10)

## <a id='3-4'>3.4 2D Histogram</a>

In [ ]:
data = [
    go.Histogram2d(
        x=hits.x,
        y=hits.y
    )
]
iplot(data)

## <a id='3-5'>3.5 3D Surface plot</a>

In [ ]:
hits_sample = hits.sample(8000)
cols_to_use = ['x','y','z']
hits_sample[cols_to_use].iplot(kind='surface',colorscale='rdylbu')

## <a id='3-6'>3.6 3D Sactter plot</a>

In [ ]:
hits_sample = hits.sample(8000)
trace1 = go.Scatter3d(
    x=hits_sample.x,
    y=hits_sample.y,
    z=hits_sample.z,
    mode='markers',
    marker=dict(
        size=6,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=1
    ),
    name = hits_sample.volume_id
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## <a id='3-7'>3.7 Horizontall Intersection ( z , y ) in Detection Layers</a>

In [ ]:
hits_samples = hits.sample(8000)

fig = ff.create_facet_grid(
    hits_samples,
    x='z',
    y='y',
    color_name='volume_id',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='3-8'>3.8 Histogram of Volume id, Layer id and Module id for single event</a>

In [ ]:
df = pd.DataFrame({'volume id': hits.volume_id,
                   'Layer id': hits.layer_id,
                   'Module id': hits.module_id})
df.iplot(kind='histogram', subplots=True, shape=(3, 1), filename='cufflinks/histogram-subplots')

## <a id='3-9'>3.9 ScatterPlotMatrix of full hits table</a>

In [ ]:
hits_samples = hits.sample(8000)

fig = ff.create_scatterplotmatrix(hits_samples[['x','y','z','volume_id']], index='volume_id', size=10, height=800, width=800)
iplot(fig, filename = 'Index a Column')

# <a id='4'>4. Event hit cells</a>
--------------------------------------------------------
The cells file contains the constituent active detector cells that comprise each hit. The cells can be used to refine the hit to track association. A cell is the smallest granularity inside each detector module, much like a pixel on a screen, except that depending on the volume_id a cell can be a square or a long rectangle. It is identified by two channel identifiers that are unique within each detector module and encode the position, much like column/row numbers of a matrix. A cell can provide signal information that the detector module has recorded in addition to the position. Depending on the detector type only one of the channel identifiers is valid, e.g. for the strip detectors, and the value might have different resolution.

## <a id='4-1'>4.1 Glimpse of Event hit cells data</a>

In [ ]:
cells.head()

- **hit_id:** numerical identifier of the hit as defined in the hits file.
- **ch0, ch1:** channel identifier/coordinates unique within one module.
- **value:** signal value information, e.g. how much charge a particle has deposited.

## <a id='4-2'>4.2 Visualization of channel identifier/coordinates unique within one module.</a>

In [ ]:
cells_samples = cells.sample(8000)
fig = ff.create_facet_grid(
    cells_samples,
    x='ch0',
    y='ch1',
#     facet_col='value',
#     color_name='value',
#     color_is_cat=True,
    color_name='value',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='4-3'>4.3 Channel identifiers (ch0, ch1) and signal value information for a single event</a>

In [ ]:
df = pd.DataFrame({'channel identifier(ch0) unique within one module': cells.ch0,
                   'channel identifier(ch1) unique within one module': cells.ch1,
                   'Signal value information': cells.value})
df.iplot(kind='histogram', subplots=True, shape=(3, 1), filename='cufflinks/histogram-subplots')

## <a id='4-4'>4.4 ScatterPlotMatrix of Event hits cells</a>  

In [ ]:
cells_samples = cells.sample(8000)

fig = ff.create_scatterplotmatrix(cells_samples, index='value', size=10, height=800, width=800)
iplot(fig, filename = 'Index a Column')

# <a id='5'>5. Event particles Data </a>

## <a id='5-1'>5.1 Glimpse of Event particles data</a>

In [ ]:
particles.head()

The particles files contains the following values for each particle/entry:

- **particle_id:** numerical identifier of the particle inside the event.
- **vx, vy, vz:** initial position or vertex (in millimeters) in global coordinates.
- **px, py, pz:** initial momentum (in GeV/c) along each global axis.
- **q:** particle charge (as multiple of the absolute electron charge).
- **nhits:** number of hits generated by this particle.

All entries contain the generated information or ground truth.

## <a id='5-2'>5.2 Initial position or vertex (in millimeters) in global coordinates. 'X' and 'Y' axis with number of hits generated by particular particle</a>

In [ ]:
particles_samples = particles.sample(10000)

fig = ff.create_facet_grid(
    particles_samples,
    x='vx',
    y='vy',
#     facet_col='nhits',
#     color_name='nhits',
#     color_is_cat=True,
    color_name='nhits',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

In [ ]:
particles_samples = particles.sample(10000)

fig = ff.create_facet_grid(
    particles_samples,
    x='vx',
    y='vy',
    facet_col='nhits',
    color_name='nhits',
    color_is_cat=True,
#     color_name='nhits',
#     show_boxes=False,
#     marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='5-3'>5.3 Distribution of particle charges in event000001005</a>

In [ ]:
temp = particles['q'].value_counts()
labels = temp.index
sizes = (temp / temp.sum())*100
trace = go.Pie(labels=labels, values=sizes, hoverinfo='label+percent')
layout = go.Layout(title='Distribution of particle charges in event000001005')
data = [trace]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## <a id='5-4'>5.4 initial position or vertex (in millimeters) in global coordinates. 'X' and 'Y' axis with particle charge(positive or negative)</a>

In [ ]:
# color set

_11_lg = '#95C061'
_12_lb = '#75ABDE'
_13_lo = '#FE9C43'
_14_lbr = '#A6886D'
_15_lr = '#E7675D'
_16_lp = '#7C5674'

color_set = [_11_lg, _12_lb, _13_lo, _14_lbr, _15_lr, _16_lp]
data = []
i = 0
for col in particles_samples['q'].unique():
    data.append(go.Scatter(x=particles_samples[particles_samples['q'] == col]['vx'], y=particles_samples[particles_samples['q'] == col]['vy'], 
                           mode='lines+markers', line=dict(color=color_set[i], width=1, dash='dash'), 
                           marker=dict(color=color_set[i], size=10), name=col))
    i += 1

layout = go.Layout(
    xaxis = dict(
        title = 'Vx',
    ),
    yaxis = dict(
        title = 'vy',
    ),
)
    
fig = go.Figure(data=data, layout=layout)

iplot(fig) 

## <a id='5-5'>5.5 Initial momentum (in GeV/c) along 'X' and 'Y' axis with particle charges(Positive or Negative).</a>

In [ ]:
particles_samples = particles.sample(10000)
data = []
i = 0
for col in particles_samples['q'].unique():
    data.append(go.Scatter(x=particles_samples[particles_samples['q'] == col]['px'], y=particles_samples[particles_samples['q'] == col]['py'], 
                           mode='lines+markers', line=dict(color=color_set[i], width=1, dash='dash'), 
                           marker=dict(color=color_set[i], size=10), name=col))
    i += 1

layout = go.Layout(
    xaxis = dict(
        title = 'Vx',
    ),
    yaxis = dict(
        title = 'vy',
    ),
)
    
fig = go.Figure(data=data, layout=layout)

iplot(fig) 

## <a id='5-6'>5.6 3D scatter plot of inintial momentum and initial vertex</a>

* Please **zoom out** to see the initial momentum (in GeV/c) along each global axis

In [ ]:
particles_samples = particles.sample(10000)

trace1 = go.Scatter3d(
    x=particles_samples.px,
    y=particles_samples.py,
    z=particles_samples.pz,
    mode='markers',
    marker=dict(
        size=6,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=1
    ),
    name = 'initial momentum (in GeV/c) along each global axis'
)
trace2 = go.Scatter3d(
    x=particles_samples.vx,
    y=particles_samples.vy,
    z=particles_samples.vz,
    mode='markers',
    marker=dict(
        size=6,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=1
    ),
    name = 'initial position or vertex (in millimeters) in global coordinates.'
)
data = [trace1, trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## <a id='5-7'>5.7 Initial position or vertex (in millimeters) in global coordinates. 'Z' and 'Y' axis with number of hits generated by particular particle</a>

In [ ]:
particles_samples = particles.sample(10000)

fig = ff.create_facet_grid(
    particles_samples,
    x='vz',
    y='vy',
#     facet_col='nhits',
#     color_name='nhits',
#     color_is_cat=True,
    color_name='nhits',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='5-8'>5.8 initial momentum (in GeV/c) along 'X' and 'Y' global axis with number of hits and physics particles</a>

In [ ]:
particles_samples = particles.sample(10000)

fig = ff.create_facet_grid(
    particles_samples,
    x='vx',
    y='vy',
    facet_row = 'q',
    facet_col = 'nhits',
#     color_name='nhits',
    color_is_cat=True,
#     color_name='nhits',
    show_boxes=False,
    marker={'size': 6, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='5-9'>5.9 Distribution of number of hits generated by particular particle</a>

In [ ]:
particles['nhits'].iplot(kind='histogram',filename='cufflinks/histogram-subplots')

# <a id='6'>6. Event truth data</a>
----------------------------------------------
The truth file contains the mapping between hits and generating particles and the true particle state at each measured hit. Each entry maps one hit to one particle.

## <a id='6-1'>6.1 Glimpse of Event truth data</a>

In [ ]:
truth.head()

- **hit_id:** numerical identifier of the hit as defined in the hits file.
- **particle_id:** numerical identifier of the generating particle as defined in the particles file. A value of 0 means that the hit did not originate from a reconstructible particle, but e.g. from detector noise.
- **tx, ty, tz :** true intersection point in global coordinates (in millimeters) between the particle trajectory and the sensitive surface.
- **tpx, tpy, tpz :** true particle momentum (in GeV/c) in the global coordinate system at the intersection point. The corresponding vector is tangent to the particle trajectory at the intersection point.
- **weight :** per-hit weight used for the scoring metric; total sum of weights within one event equals to one.

## <a id='6-2'>6.2 Vertical Intersection ( x,y ) between the particle trajectory and the sensitive surface</a>

In [ ]:
truth_samples = truth.sample(8000)

fig = ff.create_facet_grid(
    truth_samples,
    x='tx',
    y='ty',
    color_name='weight',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='6-3'>6.3 Horizontal Intersection ( z,y ) between the particle trajectory and the sensitive surface</a>

In [ ]:
truth_samples = truth.sample(8000)

fig = ff.create_facet_grid(
    truth_samples,
    x='tz',
    y='ty',
    color_name='weight',
    show_boxes=False,
    marker={'size': 10, 'opacity': 1.0, 'color': 'rgb(86, 7, 100)'},
    )
iplot(fig, filename='facet - custom colormap')

## <a id='6-4'>6.4 3D scatter plot of true particle momentum (in GeV/c) in the global coordinate system at the intersection point</a>

In [ ]:
truth_samples = truth.sample(8000)

trace2 = go.Scatter3d(
    x=truth_samples.tpx,
    y=truth_samples.tpy,
    z=truth_samples.tpz,
    mode='markers',
    marker=dict(
        size=6,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=1
    ),
)
data = [trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## <a id='6-5'>6.5 Distribution of Hit ID, Particle ID and weight for each measurement hit</a>

In [ ]:
df = pd.DataFrame({'Hit ID': truth.hit_id,
                   'Particle ID': truth.particle_id,
                   ' per-hit weight used for the scoring metric': truth.weight})
df.iplot(kind='histogram', subplots=True, shape=(3, 1), filename='cufflinks/histogram-subplots')

# <a id='7'>7. Detector Geometry Information</a>
-----------------------------------------------------------------------------
The detector is built from silicon slabs (or modules, rectangular or trapezoïdal), arranged in cylinders and disks, which measure the position (or hits) of the particles that cross them. The detector modules are organized into detector groups or volumes identified by a volume id. Inside a volume they are further grouped into layers identified by a layer id. Each layer can contain an arbitrary number of detector modules, the smallest geometrically distinct detector object, each identified by a module_id. Within each group, detector modules are of the same type have e.g. the same granularity. All simulated detector modules are so-called semiconductor sensors that are build from thin silicon sensor chips. Each module can be represented by a two-dimensional, planar, bounded sensitive surface. These sensitive surfaces are subdivided into regular grids that define the detectors cells, the smallest granularity within the detector.

Each module has a different position and orientation described in the detectors file. A local, right-handed coordinate system is defined on each sensitive surface such that the first two coordinates u and v are on the sensitive surface and the third coordinate w is normal to the surface. The orientation and position are defined by the following transformation

## <a id='7-1'>7.1 Glimpse of  Detector Geometry Information data</a>

In [ ]:
detectors.head()

- **volume_id:** numerical identifier of the detector group.
- **layer_id:** numerical identifier of the detector layer inside the group.
- **module_id:** numerical identifier of the detector module inside the layer.
- **cx, cy, cz:** position of the local origin in the described in the global coordinate system (in millimeter).
- **rot_xu, rot_xv, rot_xw, rot_yu, ...:** components of the rotation matrix to rotate from local u,v,w to global x,y,z coordinates.
- **module_t:** thickness of the detector module (in millimeter).
- **module_minhu, module_maxhu:** the minimum/maximum half-length of the module boundary along the local u direction (in millimeter).
- **module_hv:** the half-length of the module boundary along the local v direction (in millimeter).
- **pitch_u, pitch_v:** the size of detector cells along the local u and v direction (in millimeter).

## <a id='7-2'>7.2 Distribution of module_t (thickness of the detector module (in millimeter)).</a>

In [ ]:
detectors['module_t'].iplot(kind='hist', bins=10)

* As we can thickness of detector module(in millimeter) is only 0.15 , 0.25 and 0.35.

## <a id='7-3'>7.3 Distribution of module_hv ( the half-length of the module boundary along the local v direction (in millimeter)).</a>

In [ ]:
detectors['module_hv'].iplot(kind='hist', bins=10)

 * The half-length of the module boundary along the local v direction (in millimeter) is only either 36 , 56 or 78.

## <a id='7-4'>7.4 Correlation of features</a>

In [ ]:
data = [
    go.Heatmap(
        z= detectors.corr().values,
        x=detectors.columns.values,
        y=detectors.columns.values,
        colorscale='Viridis',
        reversescale = False,
        text = True ,
        opacity = 1.0 )
]

layout = go.Layout(
    title='Correlation of features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap')

## <a id='7-5'>7.5 Distribution of Layer ID, Volume ID and Module ID of detector Geometry information data</a>

In [ ]:
df = pd.DataFrame({'Volume ID(of the detector group)': detectors.volume_id,
                   'Layer ID (detector layer inside the group)': detectors.layer_id,
                   'Module ID (detector module inside the layer)': detectors.module_id})
df.iplot(kind='histogram', subplots=True, shape=(3, 1), filename='cufflinks/histogram-subplots')

## <a id='7-6'>7.6  Distribution of the size of detector cells along the local u and v direction (in millimeter)</a>

In [ ]:
df = pd.DataFrame({'the size of detector cells along the local u direction(in millimeter)': detectors.pitch_u,
                   'the size of detector cells along the local v direction(in millimeter)': detectors.pitch_v})
df.iplot(kind='histogram', subplots=True, shape=(3, 1), filename='cufflinks/histogram-subplots')

# More to come !! Stayed 
# If you find useful please upvote/leave a comment.